[View in Colaboratory](https://colab.research.google.com/github/bixbyr/LeelaZero_Analysis/blob/master/Leela_Zero_Progress.ipynb)

In [14]:
!pip install -q git+https://github.com/altair-viz/altair
import altair as alt
import pandas as pd
from bs4 import BeautifulSoup
import urllib3

alt.renderers.enable('colab')

RendererRegistry.enable('colab')

In [15]:
http = urllib3.PoolManager()
soup = BeautifulSoup(http.request('GET', 'http://zero.sjeng.org/').data, 'html.parser')

table = soup.body.table
columns = [c.string for c in table.contents[2].find_all("th")]
print(columns)

row_data = []
for r in table.find_all('tr'):
  row_contents = [c.string for c in r.find_all("td")]
  if len(row_contents) > 0:
    row_data.append(row_contents)

df = pd.DataFrame.from_records(row_data, columns=columns)
df["#"] = pd.to_numeric(df["#"])
df["Games"] = pd.to_numeric(df["Games"])
df.rename(columns={"#": "Network #"}, inplace=True)

['#', 'Upload Date', 'Hash', 'Size', 'Elo', 'Games', 'Training #']


In [16]:
chart = alt.Chart(df).mark_bar().encode(
    x="Network #",
    y="Games",
    color="Size"
)
chart

Chart({
  data:      Network #       Upload Date      Hash    Size    Elo   Games Training #
  0          144  2018-05-25 00:27  9e882e52  192x15  11286   82473    7488204
  1          143  2018-05-24 03:54  057adf2c  192x15  11252    9794    7461063
  2          142  2018-05-21 19:45  7c6588ce  192x15  11206   30393    7434908
  3          141  2018-05-20 14:59  11057dd7  192x15  11185   13366    7412196
  4          140  2018-05-20 01:21  521b0868  192x15  11142    5553    7412196
  5          139  2018-05-17 20:50  a39b64cc  192x15  11095   28474    7360230
  6          138  2018-05-17 02:39  d8214630  192x15  11043    8822    7360230
  7          137  2018-05-14 08:38  90560d92  192x15  10992   31531    7314541
  8          136  2018-05-09 16:04  b6337c69  192x15  10959   50478    7251733
  9          135  2018-05-08 00:50  05b723d0  192x15  10924   10769    7234321
  10         134  2018-05-07 11:10  2fb02cfe  192x15  10880    4544    7234321
  11         133  2018-05-07 06:38  4be6f310  192x15  10844    1496    7234321
  12         132  2018-05-06 20:26  3737d7ce  192x15  10808    3248    7211113
  13         131  2018-05-04 07:45  ecab83bb  192x15  10745   31498    7175793
  14         130  2018-05-02 03:06  18e6a6c5  192x15  10710   61580    7100021
  15         129  2018-04-29 09:00  cfb2ce28  192x15  10655   53716    7062848
  16         128  2018-04-25 07:23  158603eb  192x15  10604   62391    7003809
  17         127  2018-04-22 14:49  3f6c8dd8  192x15  10574   49315    6948814
  18         126  2018-04-21 00:11  59bb7337  192x15  10548   10886    6916674
  19         125  2018-04-18 04:43  e8601c38  192x15  10511   90912    6853079
  20         124  2018-04-16 22:11  8a045bce  192x15  10487   39905    6807268
  21         123  2018-04-14 11:38  f3231682  192x15  10425   64768    6742959
  22         122  2018-04-13 22:02  c0cb605b  192x15  10394   16730    6742959
  23         121  2018-04-12 01:36  5d6d9c5b  192x15  10343   33171    6698902
  24         120  2018-04-10 10:50  8c67ecdc  192x15  10318   21462    6677575
  25         119  2018-04-09 21:12  5838db67  192x15  10281    6320    6677575
  26         118  2018-04-09 18:54  02e663b0  192x15  10244    4338    6677575
  27         117  2018-04-09 10:35  ba748d40  192x15  10214    7851    6642917
  28         116  2018-04-04 09:37  39d46507  128x10   9888  225231    6435001
  29         115  2018-04-03 19:09  8ed44722  128x10   9851   28593    6412632
  ..         ...               ...       ...     ...    ...     ...        ...
  115         29  2017-12-19 22:29  657f91c7    64x5   4244   34864    1215129
  116         28  2017-12-19 14:18  154eaebd    64x5   4168   12493    1204550
  117         27  2017-12-18 23:44  ce1aa643    64x5   4010   17064    1184723
  118         26  2017-12-18 07:50  0eb8b23b    64x5   3964   23810    1163371
  119         25  2017-12-17 15:54  808ac5f3    64x5   3808   15947    1143566
  120         24  2017-12-16 16:46  3382a7d3    64x5   3726   34833    1105000
  121         23  2017-12-16 06:25  4e4d09be    64x5   3523    7009    1095000
  122         22  2017-12-15 21:09  3ba92804    64x5   3425   16320    1080000
  123         21  2017-12-15 16:46  d11131e3    64x5   3305    6650    1070000
  124         20  2017-12-14 21:19  251b7ca8    64x5   3197   27205    1050000
  125         19  2017-12-14 15:48  e671af6f    64x5   3077    7939    1035000
  126         18  2017-12-13 06:21  4701bb9f    64x5   2923   24559    1000000
  127         17  2017-12-12 06:33  5bcb392e    64x5   2893   38353     970000
  128         16  2017-12-11 18:59  fe7a6c28    64x5   2604   20389     955000
  129         15  2017-12-11 12:07  a0d3531f    64x5   2381    9515     945000
  130         14  2017-12-11 06:43  96bf20e7    64x5   2255    7192     920000
  131         13  2017-12-09 19:13  1607dbea    64x5   1927   28131     890000
  132         12  2017-12-09 08:22  eebb910d    64x5   1732   18400     860000
  133         11  2017-12-07 13:07  